In [1]:
from face_detection import main

In [2]:
main('./images/CK+/cohn-kanade-plus-images/S005/001/S005_001_00000011.png', "", 1)

[307.13391,
 201.99915,
 -0.0032405474,
 438.5672,
 194.86722,
 3.1994605,
 259.87903,
 178.73419,
 11.788714,
 341.62595,
 174.00339,
 -25.226755,
 401.3927,
 171.53769,
 -23.705614,
 479.63004,
 168.82491,
 16.375118,
 372.52872,
 199.30902,
 -24.594208,
 376.36417,
 271.49146,
 -56.299461,
 376.41443,
 311.95346,
 -24.950884,
 379.43185,
 357.60095,
 -12.843111,
 330.94168,
 333.15857,
 7.9911337,
 425.30103,
 329.61536,
 10.754564,
 377.99237,
 332.6976,
 -14.308941,
 410.32751,
 277.12994,
 -7.0908031,
 342.28757,
 278.16431,
 -9.1516829,
 375.8862,
 291.78452,
 -26.689037,
 310.23154,
 195.10071,
 -8.9472837,
 334.14941,
 203.57199,
 1.155954,
 307.37128,
 209.27304,
 -0.58583272,
 284.04236,
 204.14308,
 11.06298,
 308.60532,
 203.3325,
 -3.3958294,
 433.574,
 189.92918,
 -5.8527708,
 458.24335,
 196.53714,
 15.098604,
 437.76068,
 204.03271,
 2.5838068,
 411.06549,
 200.5092,
 3.0681994,
 434.95612,
 198.1026,
 -0.39119533,
 303.67923,
 160.28976,
 -17.755552,
 437.48163,
 154.

In [16]:
#!/usr/bin/python

import os
from os import listdir
from cd import cd

def get_all_folders(folder_path):
    with cd(folder_path):
        all_folders = os.listdir('.')
        directory_names = []
        for directory in all_folders:
            if directory[0] == '.':
                continue
            else:
                directory_names.append(directory)
        return directory_names
    
labels = list()
features = list()

def grab_data(Folder):
    counter = 0
    
    ck_path = os.getcwd() + '/images/CK+/' + Folder + '/'
    s_folders = get_all_folders(ck_path)
    for folder in s_folders:
        subdir_path = ck_path + folder + '/'
        zero_folders = get_all_folders(subdir_path)
        for zero in zero_folders:
            img_level_path = subdir_path + zero + '/'
            imgs = get_all_folders(img_level_path)
            if Folder == 'Emotions': #LABELS
                if len(imgs) > 0: #has an emotion label
                    last_img_path = img_level_path + imgs[-1]
                    f = open(last_img_path, 'r')
                    emotion = f.read()
                    labels.append(emotion[3]) #emotion[3] is the number denoting emotion
                else: #has no emotion label
                    labels.append(-1)
            else: #FEATURES
                last_img_path = img_level_path + imgs[-1]
                if counter < len(labels): #TODO: CHECK THIS OFF BY ONE BULLSHIT
                    if labels[counter] != -1:
                        print counter
                        features.append(main(last_img_path, "", 1))
                counter += 1
#                 if counter > 50: #because of bad internet
#                     return

grab_data('Emotions')
grab_data('cohn-kanade-plus-images')
temp_labels = list()
for label in labels: #clean temp labels
    if label != -1:
        temp_labels.append(label)
labels = temp_labels


0
2
4
6
7
8
9
10
11
12
13
14
15
17
18
22
23
24
25
26
29
30
31
32
34
35
36
37
38
40
42
43
47
48
49
51
54
55
56
58
60
61
63
66
70
71
72
73
74
75
77
78
80
82
84
85
86
89
90
91
93
95
96
97
98
100
101
102
103
105
106
107
108
110
113
114
117
118
119
120
123
124
125
126
127
128
129
130
132
133
134
135
136
138
139
141
142
143
144
146
147
148
149
151
153
154
155
157
158
159
160
162
163
164
167
168
169
170
171
173
174
175
180
181
182
187
188
189
191
192
194
197
198
200
201
205
206
207
212
214
217
220
221
222
224
225
229
230
231
232
236
237
241
243
246
248
249
251
252
253
254
255
256
259
260
262
263
266
267
268
269
270
271
272
273
275
276
278
279
280
283
284
286
287
292
293
294
295
297
298
299
302
304
307
308
309
312
313
314
317
318
320
325
326
332
349
351
353
355
357
361
366
367
369
372
374
375
376
383
388
389
394
395
397
402
403
408
409
412
416
417
422
423
424
426
429
437
439
444
452
456
458
461
462
463
467
468
469
470
471
472
473
476
477
479
480
481
482
484
485
486
492
494
495
496
497
499
502


In [84]:
import numpy as np

labels = np.array(labels).astype(np.float) #does it matter if the labels are strings or floats? what's the difference?
#print labels

features = np.array(features).astype(np.float)
#print features[:1]

[ 3.  7.  1.  5.  7.  6.  4.  1.  3.  5.  7.  6.  1.  5.  7.  1.  3.  7.
  6.  1.  5.  1.  1.  7.  1.  4.  3.  5.  7.  1.  5.  7.  3.  5.  7.  1.
  5.  7.  6.  1.  5.  7.  5.  3.  3.  1.  6.  7.  4.  3.  4.  7.  1.  5.
  7.  3.  7.  5.  3.  7.  5.  4.  7.  3.  7.  3.  1.  5.  4.  3.  7.  5.
  7.  3.  5.  7.  1.  3.  7.  4.  5.  7.  3.  7.  5.  3.  4.  7.  5.  3.
  7.  5.  7.  5.  6.  4.  7.  5.  3.  7.  5.  6.  1.  7.  1.  5.  3.  5.
  7.  4.  3.  7.  3.  5.  7.  5.  3.  7.  6.  1.  5.  3.  1.  5.  7.  3.
  4.  7.  3.  5.  7.  3.  5.  1.  7.  3.  5.  7.  3.  7.  5.  3.  7.  3.
  5.  7.  6.  3.  7.  6.  3.  7.  1.  3.  5.  7.  4.  5.  7.  3.  7.  5.
  7.  3.  5.  1.  7.  3.  7.  5.  1.  7.  3.  1.  4.  5.  7.  1.  5.  6.
  5.  7.  5.  7.  3.  5.  6.  7.  3.  5.  7.  3.  5.  3.  5.  7.  5.  3.
  7.  1.  5.  7.  7.  4.  3.  3.  6.  3.  5.  7.  3.  6.  3.  5.  1.  3.
  5.  7.  7.  1.  3.  1.  7.  6.  1.  7.  5.  7.  6.  5.  7.  3.  5.  7.
  4.  1.  7.  4.  1.  7.  7.  4.  3.  5.  6.  5.  4

In [79]:
#sanity check
print len(features)
print len(labels)


327
327


In [90]:
#tools
def train_test_split(split): 
    features_train = features[:split]
    labels_train = labels[:split]
    features_test = features[split:]
    labels_test = labels[split:]
    return (features_train, labels_train, features_test, labels_test)

def num_of_label(val, labels):
    count = 0
    for label in labels:
        if label == val:
            count += 1
    return count

def print_accuracy(labels_actual, labels_predicted):
    correct = 0
    wrong = 0
    for index in range(len(labels_actual)):
        if labels_actual[index] == labels_predicted[index]:
            correct += 1
        else:
            wrong += 1
    print 'num correct: ' + str(correct)
    print 'num wrong: ' + str(wrong)
    print 'accuracy: ' + str(float(correct)/(correct + wrong))

In [117]:
from sklearn import svm

#X is a matrix of input examples [[x1.....xn],...[x1,..., xn]]
#Y is a vector of labels corresponding to emotion [1, 2, 8, ... yn]


def train(X, y):
#    clf = svm.SVC(decision_function_shape='ovo')
    clf = svm.LinearSVC()
    clf.fit(X,y)
    print 'Finished Training SVM'
    return clf

def test(clf, X):
    #Takes all examples to predict and returns an array of lables
    predictions = clf.predict(X)
    print 'Finished Predictions for SVM'
    return predictions

############ Sample usage ####################

#X = [[0,1], [3,4], [6,2], [1,2]]
#y = [1,2,3,4]
    
splits = train_test_split(250)
features_train = splits[0]
labels_train = splits[1]
features_test = splits[2]
labels_test = splits[3]

clf = train(features_train, labels_train)
predictions = test(clf, features_test)

print "training labels: "
print labels_train

for val in range (0, 8):
    print "TRAIN: num of label " + str(val) + ": " + str(num_of_label(val, labels_train))

print "predictions: "
print predictions
print "actual: "
print labels_test

# accuracy = "105% and rising!"
# print "accuracy:"
# print accuracy

print_accuracy(labels_test, predictions)

Finished Training SVM
Finished Predictions for SVM
training labels: 
[ 3.  7.  1.  5.  7.  6.  4.  1.  3.  5.  7.  6.  1.  5.  7.  1.  3.  7.
  6.  1.  5.  1.  1.  7.  1.  4.  3.  5.  7.  1.  5.  7.  3.  5.  7.  1.
  5.  7.  6.  1.  5.  7.  5.  3.  3.  1.  6.  7.  4.  3.  4.  7.  1.  5.
  7.  3.  7.  5.  3.  7.  5.  4.  7.  3.  7.  3.  1.  5.  4.  3.  7.  5.
  7.  3.  5.  7.  1.  3.  7.  4.  5.  7.  3.  7.  5.  3.  4.  7.  5.  3.
  7.  5.  7.  5.  6.  4.  7.  5.  3.  7.  5.  6.  1.  7.  1.  5.  3.  5.
  7.  4.  3.  7.  3.  5.  7.  5.  3.  7.  6.  1.  5.  3.  1.  5.  7.  3.
  4.  7.  3.  5.  7.  3.  5.  1.  7.  3.  5.  7.  3.  7.  5.  3.  7.  3.
  5.  7.  6.  3.  7.  6.  3.  7.  1.  3.  5.  7.  4.  5.  7.  3.  7.  5.
  7.  3.  5.  1.  7.  3.  7.  5.  1.  7.  3.  1.  4.  5.  7.  1.  5.  6.
  5.  7.  5.  7.  3.  5.  6.  7.  3.  5.  7.  3.  5.  3.  5.  7.  5.  3.
  7.  1.  5.  7.  7.  4.  3.  3.  6.  3.  5.  7.  3.  6.  3.  5.  1.  3.
  5.  7.  7.  1.  3.  1.  7.  6.  1.  7.  5.  7.  6.  5

In [123]:
##LINEAR REGRESSION##
from sklearn import linear_model
#clf = linear_model.Ridge (alpha = .5)
clf = linear_model.LinearRegression()
clf.fit(features_train, labels_train)
# print("coefficient: ")
# clf.coef_
# print("predict: ")
test_results = clf.predict(features_test)
updated_results = list()
for result in test_results:
    updated_results.append(int(round(result)))
updated_results = np.array(updated_results).astype(np.float)
print updated_results
print labels_test

print_accuracy(labels_test, updated_results)


[  4.   4.   5.   5.   3.   3.   1.   6.   6.   5.   7.   6.   6.   2.   5.
   6.   5.   3.   3.   4.   4.   5.   3.   4.   5.   2.   4.   5.   4.   5.
   3.   3.   6.   5.   2.   3.  11.   6.   8.   3.   1.   2.   7.   8.   9.
   7.   6.   4.   4.   4.   5.   4.   3.   3.   3.   4.   3.   4.   5.   3.
   5.   5.   5.   5.   7.   2.   3.   7.   4.   5.   6.   4.   5.   5.   6.
   5.   2.]
[ 1.  7.  5.  1.  3.  5.  7.  1.  3.  5.  7.  1.  6.  3.  5.  7.  6.  5.
  3.  6.  4.  3.  5.  7.  1.  7.  5.  1.  5.  3.  7.  5.  7.  6.  1.  5.
  7.  6.  5.  4.  7.  5.  6.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  1.  4.  6.  1.  2.  4.  1.  2.  6.  1.  2.  4.  6.  2.  6.  1.  2.
  4.  6.  2.  1.  4.]
num correct: 10
num wrong: 67
accuracy: 0.12987012987


In [127]:
##RANDOM##
import random

num_examples = len(predictions)
random_labels = [random.randint(0,7) for p in range(0,num_examples)]
# print random_labels
# print labels_test
print_accuracy(labels_test, random_labels)

num correct: 12
num wrong: 65
accuracy: 0.155844155844


429
437
439
444
452
456
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-14-258d5e0ae85f> in <module>()
     46 
     47 grab_data('Emotions')
---> 48 grab_data('cohn-kanade-plus-images')
     49 temp_labels = list()
     50 for label in labels: #clean temp labels

<ipython-input-14-258d5e0ae85f> in grab_data(Folder)
     42                 if labels[counter] != -1:
     43                     print counter
---> 44                     features.append(main(last_img_path, "", 1))
     45                 counter += 1
     46 

/Users/Cheson/Documents/CS229Final/cs229_emotion_detection/face_detection.py in main(input_filename, output_filename, max_results)
    110 def main(input_filename, output_filename, max_results):
    111     with open(input_filename, 'rb') as image:
--> 112         faces = detect_face(image, max_results)
    113         #print('Found %s face%s' % (len(faces), '' if len(faces) == 1 else 's'))
    114 

/Users/Cheson/Documents/CS229Final/cs229_emotion_detection/face_detection.py in detect_face(face_file, max_results)
     65         })
     66     response = request.execute()
---> 67     return response['responses'][0]['faceAnnotations']
     68 # [END detect_face]
     69 

KeyError: 'faceAnnotations'

In [ ]:

from sklearn import svm
​
#X is a matrix of input examples [[x1.....xn],...[x1,..., xn]]
#Y is a vector o